# Trabalhando com SQL no Python

### Baixando o arquivo da aula:

In [11]:
import gdown

# URL do Google Drive convertida para download direto
url = 'https://drive.google.com/uc?id=1kcH4xji_A1_FCgHoDOf6eu9EYcX0blyP'

# Caminho local para salvar o arquivo .db
output_path = 'chinook.db'

# Baixar o arquivo
gdown.download(url, output_path, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1kcH4xji_A1_FCgHoDOf6eu9EYcX0blyP
To: /content/chinook.db
100%|██████████| 1.07M/1.07M [00:00<00:00, 101MB/s]


'chinook.db'

Chinook é um banco de dados de exemplo disponível para SQL Server, Oracle, MySQL, entre outros. Ele pode ser criado executando um único script SQL. O banco de dados Chinook é uma alternativa ao banco de dados Northwind, sendo ideal para demonstrações e testes de ferramentas ORM que visam servidores de banco de dados únicos e múltiplos.

### SQLite + Pandas

Para trabalhar com o banco de dados em Python, primeiro use a função `connect` do `sqlite3` para se conectar ao banco de dados e obter um objeto `Connection`:

In [12]:
import sqlite3
connection = sqlite3.connect('chinook.db')


<img src="https://docs.yugabyte.com/images/sample-data/chinook/chinook-er-diagram.png" alt="Chinook sample database">

O pandas oferece um método eficiente para carregar dados de bancos de dados SQL. Geralmente, utilizamos esse método para executar uma consulta SQL, usando uma conexão já estabelecida com o banco de dados. Assim, para visualizar todas as tabelas do banco podemos utilizar a consulta:

In [13]:
import pandas as pd
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type = 'table'", connection)

,type,name,tbl_name,rootpage,sql
0,table,Album,Album,2,CREATE TABLE [Album]\n(\n [AlbumId] INTEGER...
1,table,Artist,Artist,3,CREATE TABLE [Artist]\n(\n [ArtistId] INTEG...
2,table,Customer,Customer,4,CREATE TABLE [Customer]\n(\n [CustomerId] I...
3,table,Employee,Employee,7,CREATE TABLE [Employee]\n(\n [EmployeeId] I...
4,table,Genre,Genre,9,CREATE TABLE [Genre]\n(\n [GenreId] INTEGER...
5,table,Invoice,Invoice,10,CREATE TABLE [Invoice]\n(\n [InvoiceId] INT...
6,table,InvoiceLine,InvoiceLine,12,CREATE TABLE [InvoiceLine]\n(\n [InvoiceLin...
7,table,MediaType,MediaType,14,CREATE TABLE [MediaType]\n(\n [MediaTypeId]...
8,table,Playlist,Playlist,15,CREATE TABLE [Playlist]\n(\n [PlaylistId] I...
9,table,PlaylistTrack,PlaylistTrack,16,CREATE TABLE [PlaylistTrack]\n(\n [Playlist...


O banco de dados possui onze tabelas. Analisando a tabela de Album, temos:

In [14]:
pd.read_sql("SELECT * FROM Album", connection)


,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


Perceba que temos duas colunas de índice. Para resolver isso:

In [15]:
pd.read_sql("select * from Album", connection, index_col=["AlbumId"])

,Title,ArtistId
AlbumId,,
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3
...,...,...
343,Respighi:Pines of Rome,226
344,Schubert: The Late String Quartets & String Qu...,272
345,Monteverdi: L'Orfeo,273


Analisando outra tabela do banco:

In [17]:
pd.read_sql("SELECT * FROM Invoice", connection).head(3)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94


Vamos criar uma função chamada sq para simplificar o nosso trabalho. Com ela não vamos precisar ficar escrevendo pd.read_sql a todo momento...

In [29]:
def sq(str, con=connection):
  return pd.read_sql("""{}""", format(str), con)

Frequentemente selecionaremos linhas em um banco de dados que atendem a certos critérios de seleção, especialmente em grandes volumes de dados, onde um banco de dados pode conter muitas linhas. Apenas as linhas que satisfazem os critérios de seleção (formalmente chamados de predicados) são selecionadas. A cláusula WHERE do SQL especifica os critérios de seleção de uma consulta. Valores de string em consultas SQL são delimitados por aspas simples (').

In [31]:
pd.read_sql("select * from Invoice where total > '10'", connection)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
1,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
2,19,40,2009-03-14 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,13.86
3,26,19,2009-04-14 00:00:00,1 Infinite Loop,Cupertino,CA,USA,95014,13.86
4,33,57,2009-05-15 00:00:00,"Calle Lira, 198",Santiago,None,Chile,None,13.86
...,...,...,...,...,...,...,...,...,...
59,383,10,2013-08-12 00:00:00,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,13.86
60,390,48,2013-09-12 00:00:00,Lijnbaansgracht 120bg,Amsterdam,VV,Netherlands,1016,13.86
61,397,27,2013-10-13 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,13.86
62,404,6,2013-11-13 00:00:00,Rilská 3174/6,Prague,None,Czech Republic,14300,25.86


Complicando um pouco:

In [32]:
pd.read_sql("select * from invoice where total < (select avg(total) from invoice)", connection)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,6,37,2009-01-19 00:00:00,Berger Straße 10,Frankfurt,None,Germany,60316,0.99
3,7,38,2009-02-01 00:00:00,Barbarossastraße 19,Berlin,None,Germany,10779,1.98
4,8,40,2009-02-01 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
...,...,...,...,...,...,...,...,...,...
228,405,20,2013-11-21 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,0.99
229,406,21,2013-12-04 00:00:00,801 W 4th Street,Reno,NV,USA,89503,1.98
230,407,23,2013-12-04 00:00:00,69 Salem Street,Boston,MA,USA,2113,1.98
231,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96


A cláusula WHERE pode conter os operadores <, >, <=, >=, =, <> (diferente) e LIKE. O operador LIKE é usado para correspondência de padrões—procurando por strings que combinam com um padrão dado. Um padrão que contém o caractere curinga de porcentagem (%) procura por strings que tenham zero ou mais caracteres na posição do caractere de porcentagem no padrão. Por exemplo, vamos localizar todos os artistas cujo nome começa com a letra D:

In [37]:
pd.read_sql(" select * from Artist where name like 'D%' ", connection)

,ArtistId,Name
0,55,David Coverdale
1,58,Deep Purple
2,78,Def Leppard
3,79,Dennis Chambers
4,80,Djavan
5,157,Dread Zeppelin
6,164,Dhani Harrison & Jakob Dylan
7,192,DJ Dolores & Orchestra Santa Massa


A cláusula ORDER BY ordena os resultados de uma consulta em ordem ascendente (do menor para o maior) ou descendente (do maior para o menor), especificados com ASC e DESC, respectivamente. A ordem de classificação padrão é ascendente, portanto, ASC é opcional. Vamos ordenar os títulos dos álbuns em ordem ascendente:

In [40]:
pd.read_sql(" select * from Album order by title asc ", connection)

,AlbumId,Title,ArtistId
0,156,...And Justice For All,50
1,257,20th Century Masters - The Millennium Collecti...,179
2,296,"A Copland Celebration, Vol. I",230
3,94,A Matter of Life and Death,90
4,95,A Real Dead One,90
...,...,...,...
342,8,Warner 25 Anos,6
343,334,Weill: The Seven Deadly Sins,264
344,267,Worlds,202
345,240,Zooropa,150


Você pode mesclar dados de várias tabelas, o que é referido como juntar as tabelas, com o JOIN.

In [41]:
pd.read_sql(" select * from Album join artist on artist.artistId = album.artistId ", connection)

,AlbumId,Title,ArtistId,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,1,AC/DC
1,2,Balls to the Wall,2,2,Accept
2,3,Restless and Wild,2,2,Accept
3,4,Let There Be Rock,1,1,AC/DC
4,5,Big Ones,3,3,Aerosmith
...,...,...,...,...,...
342,343,Respighi:Pines of Rome,226,226,Eugene Ormandy
343,344,Schubert: The Late String Quartets & String Qu...,272,272,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,273,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,274,274,Nash Ensemble


Para evitar a duplicação de colunas no seu resultado, você deve especificar explicitamente quais colunas deseja retornar na sua consulta e pode usar aliases para renomeá-las conforme necessário:

In [44]:
pd.read_sql(" select album.albumid, album.title, artist.artistid AS artist_id, artist.name FROM album JOIN artist ON artist.artistid = album.artistid ", connection)

,AlbumId,Title,artist_id,Name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,2,Balls to the Wall,2,Accept
2,3,Restless and Wild,2,Accept
3,4,Let There Be Rock,1,AC/DC
4,5,Big Ones,3,Aerosmith
...,...,...,...,...
342,343,Respighi:Pines of Rome,226,Eugene Ormandy
343,344,Schubert: The Late String Quartets & String Qu...,272,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,274,Nash Ensemble


**Exercício 1:** Qual funcionário tem o maior número total de clientes?

In [47]:
pd.read_sql(" SELECT e.FirstName || ' ' || e.LastName AS Employee, COUNt (c.customerid) AS Total_Customer FROM Employee AS e INNER JOIN Customer AS c ON e.EmployeeId = c.supportRepId GROUP BY 1 ORDER BY 1 DESC ", connection)

,Employee,Total_Customer
0,Steve Johnson,18
1,Margaret Park,20
2,Jane Peacock,21


**Exercício: 2** Quais são os principais clientes de acordo com o Invoice?

### Primeiro ETL

In [48]:
import requests
import sqlite3

def create_university_db(country):
    # URL da API
    url = f"http://universities.hipolabs.com/search?country={country}"

    # Fazer a requisição à API
    response = requests.get(url)
    universities = response.json()

    # Criar ou conectar ao banco de dados SQLite
    conn = sqlite3.connect('universities.db')
    c = conn.cursor()

    # Criar a tabela, se não existir
    c.execute('''
        CREATE TABLE IF NOT EXISTS universities (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            country TEXT,
            state_province TEXT,
            web_pages TEXT,
            domains TEXT
        );
    ''')

    # Inserir dados no banco de dados
    for university in universities:
        c.execute('''
            INSERT INTO universities (name, country, state_province, web_pages, domains)
            VALUES (?, ?, ?, ?, ?);
        ''', (
            university['name'],
            university['country'],
            university['state-province'],
            ', '.join(university['web_pages']),  # Convertendo listas em strings
            ', '.join(university['domains'])
        ))

    # Commitar as mudanças e fechar a conexão
    conn.commit()
    conn.close()

# Exemplo de uso
create_university_db("Brazil")


Conectando ao novo banco de dados:

In [49]:
connection = sqlite3.connect('universities.db')

Verificando as tabelas que foram geradas:

In [50]:
pd.read_sql(" SELECT * FROM sqlite_master WHERE type='table' ", connection)

,type,name,tbl_name,rootpage,sql
0,table,universities,universities,2,CREATE TABLE universities (\n id IN...
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"


Acessando o banco das universidades:

In [51]:
pd.read_sql(" SELECT * FROM universities ", connection)

,id,name,country,state_province,web_pages,domains
0,1,"Centro Universitário de Brasília, UNICEUB",Brazil,None,https://www.uniceub.br,"sempreceub.com, uniceub.br"
1,2,Centro Universitário Barao de Maua,Brazil,None,http://www.baraodemaua.br/,baraodemaua.br
2,3,Universidade Braz Cubas,Brazil,None,http://www.brazcubas.br/,brazcubas.br
3,4,Universidade Candido Mendes,Brazil,None,http://www.candidomendes.br/,candidomendes.br
4,5,Universidade Castelo Branco,Brazil,None,http://www.castelobranco.br/,castelobranco.br
...,...,...,...,...,...,...
174,175,Polytechnic University of Pernambuco,Brazil,None,https://upe.poli.br,"upe.poli.br, ecomp.poli.br, poli.br"
175,176,Cruzeiro do Sul University,Brazil,None,https://cruzeirodosul.edu.br,aluno.cruzeirodosul.edu.br
176,177,Federal Institute of Education,Brazil,None,https://ifsp.edu.br,aluno.ifsp.edu.br
177,178,Universidade Virtual do Estado de São Paulo,Brazil,None,https://univesp.br/,univesp.br


Procurando todas as universidades de Pernambuco.

In [52]:
pd.read_sql(" SELECT * FROM universities WHERE name LIKE '%Pernambuco%' ", connection)

,id,name,country,state_province,web_pages,domains
0,83,Universidade Federal de Pernambuco,Brazil,None,http://www.ufpe.br/,ufpe.br
1,90,Universidade Federal Rural de Pernambuco,Brazil,None,http://www.ufrpe.br/,ufrpe.br
2,114,Universidade Católica de Pernambuco,Brazil,None,http://www.unicap.br/,unicap.br
3,156,Universidade de Pernambuco,Brazil,None,http://www.upe.br/,upe.br
4,175,Polytechnic University of Pernambuco,Brazil,None,https://upe.poli.br,"upe.poli.br, ecomp.poli.br, poli.br"
